In [1]:
import pandas as pd
import numpy as np
import json
import ast
from datasets import load_dataset
import re
import subprocess
import threading
import requests
import time
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import os
import json
import math
import torch

from datasets import load_dataset
from typing import Any, Dict, List, Tuple

from typing import Any, Dict, List

def extract_tool_calls(content: str):
    tool_calls = []

    flag = False
    for line in content.split("\n"):
        if "<tool_call>" == line:
            flag = True
        elif "</tool_call>" == line:
            flag = False
        else:
            if flag:
                try:
                    tool_content = json.loads(line)
                except Exception as e:
                    print(e)
                    fixed_content = fix_json_string(line)
                    try:
                        tool_content = json.loads(fixed_content)
                    except json.JSONDecodeError:
                        print("json.JSONDecodeError")
                        return content

                tool_calls.append(
                    {
                        "id": f"call_{random.randint(1000, 10000)}",
                        "type": "function",
                        "function": {
                            "name": tool_content["name"],
                            "arguments": tool_content["arguments"],
                        },
                    }
                )

            flag = False

    return tool_calls

def fix_json_string(json_str: str):
    # Remove any leading or trailing whitespace or newline characters
    json_str = json_str.strip()

    # Stack to keep track of brackets
    stack = []

    # Clean string to collect valid characters
    fixed_str = ""

    # Dictionary for matching brackets
    matching_bracket = {")": "(", "}": "{", "]": "["}

    # Dictionary for the opposite of matching_bracket
    opening_bracket = {v: k for k, v in matching_bracket.items()}

    for char in json_str:
        if char in "{[(":
            stack.append(char)
            fixed_str += char
        elif char in "}])":
            if stack and stack[-1] == matching_bracket[char]:
                stack.pop()
                fixed_str += char
            else:
                # Ignore the unmatched closing brackets
                continue
        else:
            fixed_str += char

    # If there are unmatched opening brackets left in the stack, add corresponding closing brackets
    while stack:
        unmatched_opening = stack.pop()
        fixed_str += opening_bracket[unmatched_opening]

    # Attempt to parse the corrected string to ensure it’s valid JSON
    return fixed_str.replace("\'", "\"")


TASK_PROMPT = """
You are a helpful assistant.
""".strip()

TOOL_PROMPT = """
# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{tool_text}
</tools>
""".strip()

FORMAT_PROMPT = """
For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{"name": <function-name>, "arguments": <args-json-object>}
</tool_call>
""".strip()


get_weather_api = {
	"type": "function",
	"function": {
		"name": "get_current_weather",
		"description": "Get current weather at a location.",
		"parameters": {
			"type": "object",
			"properties": {
				"location": {
					"type": "str",
					"description": "The location to get the weather for",
                    "format": "City, State, Country",
				},
				"unit": {
					"type": "str",
                    "description": "The unit to return the weather in.",
					"enum": [
						"celsius",
						"fahrenheit"
					],
                    "default": "celsius"
				},
                "days": {
					"type": "str",
                    "description": "the number of days for the request.",
				}
			},
			"required": [
				"location",
                "days"
			]
		}
	}
}
def check_parameter_property(api_description, parameter_name, property_name):
    """
    Check if a parameter in an API description has a specific property.

    Args:
        api_description (dict): The API description in JSON format.
        parameter_name (str): The name of the parameter to check.
        property_name (str): The property to look for (e.g., 'format', 'default').

    Returns:
        bool: True if the parameter has the specified property, False otherwise.
    """
    parameters = api_description.get("parameters", {}).get("properties", {})
    parameter_info = parameters.get(parameter_name, {})

    return property_name in parameter_info


# Example usage

def convert_tools(tools: List[Dict[str, Any]]):
    return "\n".join([json.dumps(tool) for tool in tools])

# Helper function to create the system prompt for our model
def format_prompt(tools: List[Dict[str, Any]]):
    tool_text = convert_tools(tools)

    return (
        TASK_PROMPT
        + "\n\n"
        + TOOL_PROMPT.format(tool_text=tool_text)
        + "\n\n"
        + FORMAT_PROMPT
        + "\n"
    )

openai_format_tools = [get_weather_api]

system_prompt = format_prompt(openai_format_tools)


from openai import OpenAI

client = OpenAI(base_url="https://api.fc.archgw.com/v1", api_key="EMPTY")

# List models API
model = client.models.list().data[0].id
print("Model:", model)
messages = [
    {"role": "system", "content": system_prompt},
    # {"role": "user", "content": "can you help me check weather?"},
    {"role": "user", "content": "How is the weather in Seattle in 7 days?"},
    #{"role": "assistant", "content": "Of course!"},
    # {"role": "user", "content": "Seattle please"}
]

extra_body = {
    "temperature": 0.6,
    "top_p": 1.0,
    "top_k": 50,
    # "continue_final_message": True,
    # "add_generation_prompt": False,
    "logprobs": True,
    "top_logprobs": 10
}

resp = client.chat.completions.create(
    model="Arch-Function",
    messages=messages,
    extra_body=extra_body,
    stream = True
)







/Users/co-tran/Documents/arch/new_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model: Arch-Function


In [2]:
import json
from app.function_calling.hallucination_handler import HallucinationStateHandler
import pytest
import os

The ONNX file onnx/model.onnx is not a regular name used in optimum.onnxruntime, the ORTModel might not behave as expected.
The ONNX file onnx/model.onnx is not a regular name used in optimum.onnxruntime, the ORTModel might not behave as expected.


In [3]:
get_weather_api = {
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get current weather at a location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "str",
                    "description": "The location to get the weather for",
                    "format": "City, State",
                },
                "unit": {
                    "type": "str",
                    "description": "The unit to return the weather in.",
                    "enum": ["celsius", "fahrenheit"],
                    "default": "celsius",
                },
                "days": {
                    "type": "str",
                    "description": "the number of days for the request.",
                },
            },
            "required": ["location", "days"],
        },
    },
}
function_description = get_weather_api["function"]
if type(function_description) != list:
    function_description = [get_weather_api["function"]]

hallu = HallucinationStateHandler(response_iterator=resp, apis = function_description)


In [13]:
r = next(hallu.response_iterator)
if hasattr(r.choices[0].delta, "content"):
    token_content = r.choices[0].delta.content
    if token_content:
        logprobs = [p.logprob for p in r.choices[0].logprobs.content[0].top_logprobs]
        print(hallu.check_token_hallucination(token_content, logprobs), hallu.tokens)

False


In [4]:
for token in hallu:
    print(f"Token: {token}")
    print(f"Hallucination: {hallu.hallucination}")
    print(f"Hallucination Message: {hallu.hallucination_message}")

# Access the tokens processed

Token: None
Hallucination: False
Hallucination Message: 
Token: <tool_call>
Hallucination: False
Hallucination Message: 
Token: 

Hallucination: False
Hallucination Message: 
Token: {"
Hallucination: False
Hallucination Message: 
Token: name
Hallucination: False
Hallucination Message: 
Token: ":
Hallucination: False
Hallucination Message: 
function name entered
Token:  "
Hallucination: False
Hallucination Message: 
Token: get
Hallucination: False
Hallucination Message: 
Token: _current
Hallucination: False
Hallucination Message: 
Token: _weather
Hallucination: False
Hallucination Message: 
Token: ",
Hallucination: False
Hallucination Message: 
Token:  "
Hallucination: False
Hallucination Message: 
Token: arguments
Hallucination: False
Hallucination Message: 
Token: ":
Hallucination: False
Hallucination Message: 
Token:  {"
Hallucination: False
Hallucination Message: 
Token: location
Hallucination: False
Hallucination Message: 
Token: ":
Hallucination: False
Hallucination Message: 
Toke

In [9]:
for token in resp:
    if hasattr(token.choices[0].delta, "content"):
        
        token_content = token.choices[0].delta.content
        print(token_content)

In [6]:
hallu.token_probs_map

[('<tool_call>', 0.0034013802651315928, -0.0032856864854693413),
 ('Seattle', 1.1522446584422141e-05, -1.1521117812662851e-05),
 ('7', 1.553274159959983e-05, -1.5530327800661325e-05)]